In [2]:
work_dir = '/content/drive/MyDrive/logistics__yolo'

In [3]:
!pip install ultralytics -q

In [4]:
!pip install datasets[vision] -q

In [5]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import yaml
from PIL import Image
import os
import seaborn as sns
from ultralytics import YOLO
from matplotlib.patches import Rectangle
import glob
import cv2
import os
import yaml
import shutil
import random
import ultralytics
from tqdm.notebook import tqdm

In [6]:
def log(msg):
    print(f'🔔---| {msg} ')

In [7]:
#create file paths for the training and validation sets
train_path_img = "/content/drive/MyDrive/logistics project/data_from_roboflow/train/images"
train_path_label = "/content/drive/MyDrive/logistics project/data_from_roboflow/train/labels"
val_path_img = "/content/drive/MyDrive/logistics project/data_from_roboflow/valid/images/"
val_path_label = "/content/drive/MyDrive/logistics project/data_from_roboflow/valid/labels/"
test_path_img = "/content/drive/MyDrive/logistics project/data_from_roboflow/test/images"
test_path_label = "/content/drive/MyDrive/logistics project/data_from_roboflow/test/labels"

def get_random(validation_path):
    #get a random image from the validation set
    random_indexes  = random.sample(range(0, len(os.listdir(validation_path))), k=10)
    # create a folder inside data/waste_data/ to store test images and labels
    os.makedirs("data/waste_data/test/images", exist_ok=True)
    os.makedirs("data/waste_data/test/labels", exist_ok=True)
    # move the random image and labels to the test folder
    for i in random_indexes:
        image = os.listdir(validation_path)[i]
        shutil.move(validation_path + image, test_path_img + image)
        label = image.replace(".jpg", ".txt")
        shutil.move(val_path_label + label, test_path_label + label)
        log(f"Moved {image} and {label} to test folder")


In [8]:
# count the number of files in the training and validation sets
train_files = os.listdir(train_path_img)
val_files = os.listdir(val_path_img)
log(f"Number of training images: {len(train_files)}")
log(f"Number of validation images: {len(val_files)}")
log(f"Number of test images: {len(os.listdir(test_path_img))}")


🔔---| Number of training images: 2707 
🔔---| Number of validation images: 173 
🔔---| Number of test images: 124 


In [9]:
data_path = '/content/drive/MyDrive/logistics project/data_from_roboflow/data.yaml'

In [10]:
import torch
# torch version
log(f"Pytorch version: {torch.__version__}")

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
log(f"Device: {device}")

🔔---| Pytorch version: 2.2.1+cu121 
🔔---| Device: cpu 


In [11]:
os.environ['KMP_DUPLICATE_LIB_OK']='True' # for transformer models

In [12]:
model = ultralytics.YOLO('yolov8m-obb.pt')

In [ ]:
model.train(
    data=data_path,        # dataset.yaml file
    task='detect',              # 'detect', 'segment', 'class'
    imgsz=640,                  # image size
    epochs=100,                  # number of epochs
    # workers=0,                  # number of workers
    batch=16,                   # batch size
    patience=20,                # early stopping patience
    project='/content/drive/MyDrive/logistics project/runs',             # save results to project/name
    mode='train',               # 'train', 'val', 'test'
    name='logistics',     # save results to project/name
    save=True,                  # save results to project/name/weights/last.pt
                    # cuda device, i.e. 0 or 0,1,2,3 or cpu
)

Ultralytics YOLOv8.2.11 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=obb, mode=train, model=yolov8m-obb.pt, data=/content/drive/MyDrive/logistics project/data_from_roboflow/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/logistics project/runs, name=logistics5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx

train: Scanning /content/drive/MyDrive/logistics project/data_from_roboflow/train/labels.cache... 2697 images, 261 backgrounds, 8 corrupt: 100%|██████████| 2707/2707 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/logistics project/data_from_roboflow/train/images/images-2024-05-03T130636-848_jpeg.rf.00a4c4fbb6c323e0d4ebf8cbca9800fc.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0311]
train: WARNING ⚠️ /content/drive/MyDrive/logistics project/data_from_roboflow/train/images/images-2024-05-03T130636-848_jpeg.rf.231dd088e414a36a6f99678e1d205a92.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0754         1.1]
train: WARNING ⚠️ /content/drive/MyDrive/logistics project/data_from_roboflow/train/images/images-2024-05-03T130636-848_jpeg.rf.a21180f51fab99f7622bd466693cfbc2.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1639      1.2012]
train: WARNING ⚠️ /content/drive/MyDrive/logistics project/data_from_roboflow/train/images/images-2024-05-03T130636-848_jpeg.rf.a4efc2c6c1de8717eec5ddab9750a643.jpg: ignoring corrupt image/label: non-normalized or 

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/logistics project/data_from_roboflow/valid/labels.cache... 173 images, 15 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]

Plotting labels to /content/drive/MyDrive/logistics project/runs/logistics5/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 83 weight(decay=0.0), 93 weight(decay=0.0005), 92 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/drive/MyDrive/logistics project/runs/logistics5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/169 [00:00<?, ?it/s]

In [1]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/yolo_projects/dataset.yaml epochs=5 imgsz=640 batch=16 patience=20 project=/content/drive/MyDrive/yolo_projects/waste_detection name=yolo

# comment the code above and uncomment the code below to resume training if notebook is interupted
# Note: you will need to run all the previous lines of code
# !yolo task=detect mode=train resume data=/content/drive/MyDrive/Garbage_Detection/dataset.yaml model=/content/drive/MyDrive/Garbage_Detection/training_results/weights/last.pt

100% 49.7M/49.7M [00:00<00:00, 64.8MB/s]
Ultralytics YOLOv8.2.11 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/yolo_projects/dataset.yaml, epochs=5, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolo_projects/waste_detection, name=yolo, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr